In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/Shareddrives/My\ paper

/content/drive/Shareddrives/My paper


In [ ]:
import math
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

In [ ]:
data = pd.read_csv('./Customer Value Prediction/TABLES/Session Behaviors Count.csv')

In [ ]:
data.columns

Index(['MemberID', 'SessionOrder', 'add', 'checkout', 'purchase', 'register',
       'search', 'traffic', 'viewactivity', 'viewcategory', 'viewmainpage',
       'viewproduct', 'TimeDiff', 'TotalTime', 'uid_filled',
       'TotalSalesAmount', 'Active', 'VIP'],
      dtype='object')

In [ ]:
len(data)

700513

In [ ]:
data.describe()

,SessionOrder,add,checkout,purchase,register,search,traffic,viewactivity,viewcategory,viewmainpage,viewproduct,TimeDiff,TotalTime,TotalSalesAmount,Active,VIP
count,7.005130e+05,700513.000000,700513.000000,700513.000000,700513.000000,700513.000000,700513.000000,700513.000000,700513.000000,700513.000000,700513.000000,700513.000000,700513.000000,700513.000000,700513.000000,700513.000000
mean,2.318815e+06,0.166998,0.098234,0.055479,0.012821,0.062353,0.402852,0.036750,1.205088,1.284503,1.558018,13576.392959,1.747105,1377.119483,0.293521,0.161799
std,1.335184e+06,0.837909,0.635376,0.343568,0.141419,0.391526,0.490472,0.373704,3.212392,2.095138,4.266150,37707.913370,5.631009,4220.274239,0.455375,0.368266
min,7.400000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1680.000000,0.000000,0.000000
25%,1.165879e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.325449e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,701.000000,0.000000,0.000000,0.000000,0.000000
75%,3.482036e+06,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,2.000000,1.000000,7369.000000,1.000000,1380.000000,1.000000,0.000000
max,4.610624e+06,81.000000,62.000000,16.000000,41.000000,47.000000,1.000000,38.000000,182.000000,995.000000,247.000000,520331.000000,190.000000,181428.000000,1.000000,1.000000


## EDA

In [ ]:
data['Member_pre'] = data['MemberID'].shift(periods=1, fill_value='First')
data['Member_next'] = data['MemberID'].shift(periods=-1, fill_value='Last')

In [ ]:
data['Order'] = ['MIDDLE'] * len(data)

for index, row in data.iterrows():
  if row['MemberID']!=row['Member_pre']:
    data.at[index, 'Order'] = 'FIRST'
  if row['MemberID']!=row['Member_next']:
    data.at[index, 'Order'] = 'LAST'

In [ ]:
data.columns

Index(['MemberID', 'SessionOrder', 'add', 'checkout', 'purchase', 'register',
       'search', 'traffic', 'viewactivity', 'viewcategory', 'viewmainpage',
       'viewproduct', 'TimeDiff', 'TotalTime', 'uid_filled',
       'TotalSalesAmount', 'Active', 'VIP', 'Member_pre', 'Member_next',
       'Order'],
      dtype='object')

## 各行為出現在 session 中的次數

In [ ]:
bh_cols = ['add', 'checkout', 'purchase', 'register', 'search', 'traffic', 'viewactivity', 'viewcategory', 'viewmainpage', 'viewproduct']

In [ ]:
bh_dict = dict()
for bh in bh_cols:
  count = [len(data[data[bh]>0])]
  bh_dict[bh] = count

pd.DataFrame(bh_dict)

,add,checkout,purchase,register,search,traffic,viewactivity,viewcategory,viewmainpage,viewproduct
0,53893,34081,25943,8351,27634,282203,14324,233344,432097,270457


In [ ]:
bh_dict_2 = dict()
for bh in bh_cols:
  count = [data[bh].sum()]
  bh_dict_2[bh] = count

pd.DataFrame(bh_dict_2)

,add,checkout,purchase,register,search,traffic,viewactivity,viewcategory,viewmainpage,viewproduct
0,116984,68814,38864,8981,43679,282203,25744,844180,899811,1091412


## 初始機率

In [ ]:
first_df = data[data['Order']=='FIRST']

In [ ]:
dfs = []
for bh in bh_cols:
  df = pd.DataFrame(first_df[first_df[bh]>0].groupby(by='Order').size(), columns=[bh]).transpose()
  dfs.append(df)

In [ ]:
pd.concat(dfs)

Order,FIRST
add,7969
checkout,4842
purchase,3757
register,3149
search,3964
traffic,28257
viewactivity,2408
viewcategory,28395
viewmainpage,51200
viewproduct,29222
